In [44]:
# https://wikidocs.net/64703

## 01. 문자 단위 RNN(Char RNN)
---
모든 시점의 입력에 대해서 모든 시점에 대해서 출력하는 다대다 RNN을 구현하자

### 1. 문자 단위 RNN(Char RNN)  
RNN의 입출력의 단위가 단어 레벨(word-level)이 아니라 문자 레벨(character-level)로 하여 RNN을 구현한다면, 이를 문자 단위 RNN이라 부른다. RNN의 구조 자체가 달라진 것이 아니라 입,출력의 단위가 character로 바뀐 것이다.

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [46]:
# 문자 시퀀스 apple이 들어오면 pple!을 출력하는 RNN을 구현해보자
# 입력 데이터와 레이블 데이터에 대해서 문자 집합(vocabulary)를 만든다.
# 여기서 문자 집합은 중복을 제거한 집합이다

input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print(f'문자 집합의 크기: {vocab_size}')

문자 집합의 크기: 5


In [47]:
# 입력의 크기는 문자 집합의 크기로
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [48]:
# 문자 집합에 고유한 정수를 부여하자
# 문자 -> 정수
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [49]:
# 나중에 다시 예측 결과를 문자 시퀀스로 보기 위해 반대 함수를 만든다
# 정수 -> 문자
index_to_char = {v:k for k, v in char_to_index.items()}
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [50]:
# 이제 입력 데이터와 레이블 데이터 모두 각 문자들을 정수로 맵핑한다
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print('a p p l e :', x_data)
print('p p l e ! :', y_data)

a p p l e : [1, 4, 4, 3, 2]
p p l e ! : [4, 4, 3, 2, 0]


In [51]:
# 토치의 nn.RNN() 은 기본 입력값이 3차원이니 차원을 추가 해준다
x_data = [x_data]
y_data = [y_data]
print('a p p l e :', x_data)
print('p p l e ! :', y_data)

a p p l e : [[1, 4, 4, 3, 2]]
p p l e ! : [[4, 4, 3, 2, 0]]


In [52]:
# 각 정수를 원-핫 벡터로 바꿔준다
# 입력 데이터만 바꿔준다
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [53]:
# 입력 데이터와 레이블 데이터를 텐서로 바꿔준다
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print(f'훈련 데이터의 크기 : {X.shape}')
print(f'레이블의 크기 : {Y.shape}')


# 훈련 데이터의 크기 : torch.Size([1, 5, 5])
# 1 : 문장의 개수
# 5 : 단어의 개수
# 5 : 임베딩 크기

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


### 2. 모델 구현하기
---

In [54]:
# RNN을 구현해보자 마지막은 fc 계층으로 연결해 출력층으로 이용한다
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        
        # rnn 셀
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first = True)

        # 출력층
        self.fc = torch.nn.Linear(hidden_size, output_size, bias = True)

    def forward(self, x):   # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [55]:
# 클래스로 정의한 모델을 net에 저장
net = Net(input_size, hidden_size, output_size)

In [56]:
# 준비된 모델에 데이터를 넣어보자
outputs = net(X)
print(outputs.shape)

# torch.Size([1, 5, 5])
# 1 : 배치 차원
# 5 : 시점(timesteps)
# 5 : 출력의 크기

torch.Size([1, 5, 5])


In [57]:
# view를 이용해 2차원 텐서로 펼치기
print(outputs.view(-1, input_size).shape)
# input_size = vocab_size = 5

torch.Size([5, 5])


In [58]:
# 레이블 크기도 다시 확인하자
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [59]:
# 옵티마이저와 손실 함수 정의 
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [61]:
# 총 100번을 학습
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss=criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # 실제 어떻게 예측했는지 확인하기 위한 부분
    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, 'loss: ', loss.item(), 'prediction: ', result, 'true Y: ', y_data, 'prediction str: ', result_str)

0 loss:  1.3679968118667603 prediction:  [[4 4 4 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppp!!
1 loss:  1.1905289888381958 prediction:  [[4 4 4 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppp!!
2 loss:  1.076808214187622 prediction:  [[4 4 4 2 2]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppee
3 loss:  0.9789617657661438 prediction:  [[4 4 4 2 2]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppee
4 loss:  0.883773148059845 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
5 loss:  0.7888414263725281 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
6 loss:  0.6851658821105957 prediction:  [[4 4 4 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppp!!
7 loss:  0.5821504592895508 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
8 loss:  0.48215755820274353 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.3751258850097656 prediction:  [[4 4 3 2 0]] t